In [ ]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import sqlite3
import sys
import datetime
sqlite3.register_adapter(np.int64, int)
pd.options.mode.chained_assignment = None 
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# 展開所有dataframe columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

db = sqlite3.connect('./stock.db' , isolation_level=None)

In [ ]:
df_stock_id_name = pd.read_sql_query("select * from stockIdName",db)

In [ ]:
ids = df_stock_id_name['id'].tolist()
df_stock_id_name.head()

In [ ]:
def workdays(d, end, cformat="%Y%m%d"):
    days = []
    excluded=(6, 7)
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d.strftime(cformat)) 
        d += datetime.timedelta(days=1)
    return days

def download(date):
    # 下載股價
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date + '&type=ALL')
    
    # 整理資料，變成表格
    df_origin = pd.read_csv(StringIO(r.text.replace("=", "")), 
                header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    
    # 整理一些字串：
    df_origin = df_origin.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
    
    # 清資料
    try:
        del df_origin['證券名稱']
        del df_origin['Unnamed: 16']
    except:
        print('沒有"證券名稱"欄位')

    df = df_origin[df_origin["證券代號"].astype(str).str.match(r"^\d{4}.0$")]  # 證券代號: xxxx (上市)
    df['證券代號'] = df['證券代號'].apply(lambda id: int(id))
    
    # 合併 df, df_stock_id_name
    df = df.merge(df_stock_id_name, left_on="證券代號",right_on="id", how="left")
    
    return df

def download_otc(date):
    r = requests.get(f'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&d={date}&se=AL')
    j = r.json()
    df = pd.DataFrame.from_dict(j["aaData"])
    df.columns=['代號','名稱','收盤','漲跌','開盤','最高','最低','成交股數','成交金額(元)','成交筆數','最後買價','最後買量(千股)','最後賣價','最後賣量(千股)','發行股數','次日漲停價','次日跌停價']
    df = df[df["代號"].astype(str).str.match(r"^\d{4}$")]  
    return df


In [ ]:
today = datetime.date.today().strftime("%Y%m%d")
today_otc = datetime.date.today().strftime("%Y/%m/%d")

def start_download_sii(start=today, end=today):
    startDate = datetime.datetime.strptime(start, "%Y%m%d")
    endDate =  datetime.datetime.strptime(end, "%Y%m%d")

    for date in workdays(startDate, endDate):
        print('download sii: ', date)

        # download and insert to db
        try:
            df = download(date)
            
            for index, row in df.iterrows(): 
                sql_insert = f'insert into daily (date, id, name, tradeVolumn,[transaction], tradeValue,open,high,low,close,dir,change,bidPrice,bidVolumn,askPrice,askVolumn, pe) \
                               values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'
                db.execute(sql_insert, (date, row['證券代號'],row['name'],row['成交股數'],row['成交筆數'],row['成交金額'],row['開盤價'],row['最高價'],row['最低價'],row['收盤價'],row['漲跌(+/-)'],row['漲跌價差'],row['最後揭示買價'],row['最後揭示買量'],row['最後揭示賣價'],row['最後揭示賣量'],row['本益比']))
        except:
            print('skip...', date)
            continue

def atof(v):
    return locale.atof(v)

def start_download_otc(start=today_otc, end=today_otc):
    startDate = datetime.datetime.strptime(start, "%Y/%m/%d")
    endDate =  datetime.datetime.strptime(end, "%Y/%m/%d")

    for date in workdays(startDate, endDate, "%Y/%m/%d"):
        # 2021 -> 110
        fdate = date.split("/")
        fdate[0] = str(int(fdate[0]) - 1911)
        fdate = "/".join(fdate)
        print('download otc: ', fdate) 
        try:
            df = download_otc(fdate)
            for index, row in df.iterrows():
                sql_insert = f'insert into daily (date, id, name, tradeVolumn,[transaction], tradeValue,open,high,low,close,dir,change,bidPrice,bidVolumn,askPrice,askVolumn, pe) \
                               values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'
                change = row['漲跌']
                dir = -1.0 if ('-' in change) else 1.0 if ('+' in change) else np.nan
                change = change.replace('-', '') if ('-' in change) else change.replace('+', '') if ('+' in change) else np.nan

                try:
                    db.execute(sql_insert, (date.replace("/",""), row['代號'],row['名稱'],atof(row['成交股數']),atof(row['成交筆數']),atof(row['成交金額(元)']),atof(row['開盤']),\
                               atof(row['最高']),atof(row['最低']),atof(row['收盤']),dir, change, atof(row['最後買價']),atof(row['最後買量(千股)']),atof(row['最後賣價']),atof(row['最後賣量(千股)']),np.nan))
                except:
                    pass 
        except:
            print('skip: >>>>>>>>>', )
            continue


In [ ]:
start = "2023/08/11" 
end = "2023/08/14"
sii_start = start.replace("/","")
sii_end = end.replace("/","")

start_download_sii(sii_start, sii_end)
start_download_otc(start, end)

# 缺OTC 2019, 2020

download sii:  20230811
download sii:  20230814
download otc:  112/08/11
download otc:  112/08/14


In [ ]:
df = pd.read_sql_query("select * from daily where id=2330 order by date", db)
df
# df[df['date'].astype(str).str.contains("^2019", regex=True)]

,date,id,name,tradeVolumn,transaction,tradeValue,open,high,low,close,dir,change,bidPrice,bidVolumn,askPrice,askVolumn,pe
789,20230727,2330,台積電,13004888,12672,7396939592,570.0,570.0,566.0,569.0,1.0,3.0,569.0,142,570.0,294,14.46
1354,20230728,2330,台積電,19009675,19313,10814265557,569.0,573.0,565.0,567.0,-1.0,2.0,567.0,152,568.0,48,14.41
1355,20230731,2330,台積電,28409339,28425,16044638425,575.0,575.0,560.0,565.0,-1.0,2.0,564.0,79,565.0,424,14.35
1356,20230801,2330,台積電,18916866,13827,10711815419,565.0,568.0,564.0,567.0,1.0,2.0,566.0,335,567.0,209,14.41
1357,20230802,2330,台積電,34495766,34408,19394908189,567.0,569.0,558.0,561.0,-1.0,6.0,560.0,315,561.0,557,14.25
793,20230801,5483,中美晶,4432000,2862,752185000,173.5,173.5,168.0,168.5,-1.0,3.0,168.5,123,169.0,74,NaN
794,20230802,5483,中美晶,8292000,5508,1364269500,170.0,170.0,161.5,163.0,-1.0,5.5,163.0,91,163.5,27,NaN
795,20230807,5483,中美晶,4202000,2698,702213000,168.5,168.5,166.0,167.5,1.0,2.0,167.0,87,167.5,96,NaN
796,20230808,5483,中美晶,8989000,5753,1451878500,163.5,164.5,160.0,161.5,-1.0,6.0,161.5,3,162.0,195,NaN
797,20230809,5483,中美晶,4792000,3263,765568000,160.5,162.0,158.5,160.0,-1.0,1.5,159.5,55,160.0,31,NaN


In [ ]:
pd.read_sql_query("select * from daily where id=5483 and date=20200110 order by date", db)

# 更新上櫃pe(爬蟲沒有上櫃pe)

In [ ]:
# 財報
df_financial_statement = pd.read_sql_query("select * from financialStatement",db)
df_financial_statement = df_financial_statement.fillna(0)
df_financial_statement.replace('--', 0, inplace=True)

In [ ]:
df = pd.read_sql_query("select * from daily", db)
df = df.sort_values(by=['date'])

In [ ]:
# 把個股每月對應到的近四季eps記錄在dict

year = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]

def find_eps(id, quarters):
    df = df_financial_statement[df_financial_statement['id'] == id]
    return round(df[df['date'].isin(quarters)]['qeps'].sum(), 2)

def find_in_list_of_list(mylist, char):
    for sub_list in mylist:
        if char in sub_list:
            return mylist.index(sub_list) + 1
    
def getQuarters(yyyy, mm):
    start = find_in_list_of_list(year, mm) - 1 + 4
    res = []
    
    for i in range(4):
        if(start % 4):
            res.append(f"{yyyy}q{start % 4}")
        else:
            yyyy -= 1
            res.append(f"{yyyy}q4")
            
        start -= 1
    
    return res

dict = {}
currentYears = 2023
currentMonth = 5
for id in set(df['id'].tolist()):
    dict[id] = {}
    for yyyy in range(2017,currentYears + 1):
        for mm in range(1,currentMonth + 1):
            quarters = getQuarters(yyyy, mm)
            eps = find_eps(id, quarters)
            key = f"{yyyy}{str(mm).zfill(2)}"
            dict[id][key] = eps

In [ ]:
# getQuarters(2023, 1) # ['2022q4', '2022q3', '2022q2', '2022q1'] 用前四季eps總和
# dict[5425]

In [ ]:
# 找daily中 id在filter內 與 pe為NaN
filter_ids = [5347, 6182, 8938, 3264, 5425, 5483, 3611, 6509, 8155, 6770, 1342, 2640, 6146, 6263, 8109, 5009, 6691, 3265]

update_df = df[(df['id'].isin(filter_ids))& (df['pe'].isna())] 
update_df

In [ ]:
for index, row in update_df.iterrows():
    date = str(row['date'])[0:6]
    id = row['id']
    try:
        eps = dict[id][date]
        update_df.loc[index, 'eps'] = eps
    except: # dict沒有這季eps先拿舊的(會導致之後需要更新eps)
        preDate = list(dict[id].keys())[-1]
        eps = dict[id][preDate]
        update_df.loc[index, 'eps'] = eps

In [ ]:
def writeDB(dff):
    for index, row in dff.iterrows():
        date = row['date']
        pe = row['pe']
        id = row['id']
        sql = f"UPDATE daily SET pe = {pe} WHERE id = {id} and date = {date}"
        db.execute(sql)
        
a = update_df.sort_values(by=['date'])
a['pe'] = round(a['close']/a['eps'], 2)
writeDB(a)

In [ ]:
# search db
id = 5425
df1 = pd.read_sql_query(f"select * from daily where id={id} order by date", db)
df1

# OTC API return columns


|代號|名稱|收盤|漲跌|開盤|最高|最低|成交股數|成交金額(元)|成交筆數|最後買價|最後買量(千股)|最後賣價|最後賣量(千股)|發行股數|次日漲停價|次日跌停價|
|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|
|id|name|close|change|open|high|low|tradeVolumn|tradeValue|transaction|bidPrice|bidVolumn|askPrice|askVolumn|


In [ ]:
a = download_otc("107/01/01")
a

# Search DB 

In [ ]:
# find duplicate
df = pd.read_sql_query("select * from daily", db)
a = df[(df['id']==5425)].sort_values(by=['date'], ascending=False)
duplicated = a[a['date'].duplicated()]['date'].tolist()
print(f"dulicate: {duplicated}")

In [ ]:
# delete row
for date in duplicated:
    sql = f"delete from daily where date={date}"
    db.execute(sql)

In [ ]:
db.close()